In [27]:
import numpy as np
from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment
from Bio.SeqRecord import SeqRecord
import tqdm

In [28]:
# Open the fasta file
nter_names, cter_names, cc_names = set(), set(), set()
with open("data/cat_all_n-ter.afa", "r") as f:
    nter_al = AlignIO.read(f, "fasta")
    for rec in nter_al:
        id_short = rec.id.split(":")[0]
        rec.id = id_short
        nter_names.add(rec.id)

with open("data/c_ter.afa", "r") as f:
    cter_al = AlignIO.read(f, "fasta")
    for rec in cter_al:
        id_short = rec.id.split(":")[0]
        rec.id = id_short
        cter_names.add(rec.id)

with open("data/coiled_coil.afa", "r") as f:
    cc_al = AlignIO.read(f, "fasta")
    for rec in cc_al:
        id_short = rec.id.split(":")[0]
        rec.id = id_short
        cc_names.add(rec.id)

In [29]:
print(len(nter_al), len(cc_al), len(cter_al))

3548 2763 2495


In [30]:
def get_record(alignment, rec_id):
    for rec in alignment:
        if rec.id.startswith(rec_id):
            return rec
    return None

In [31]:
# Concatenate N-Terminal alignment with Coiled-Coil alignment
cat_nter_cc = MultipleSeqAlignment([])
already_visited = set()
cc_len = cc_al.get_alignment_length()
for rec in tqdm.tqdm(nter_al):
    if not rec.id in already_visited:
        if rec.id in cc_names:
            add_rec = get_record(cc_al, rec.id)
            cat_seq = rec.seq + add_rec.seq
            cat_rec = SeqRecord(seq=cat_seq, id=rec.id)
        else:  # the sequence from N-Ter aln is not in Coiled-coil alignment
            cat_seq = rec.seq + "".join(["-" for i in range(cc_len)])  # add gaps '-' everywhere in the CC alignment because seq doesn't have this domain
            cat_rec = SeqRecord(seq=cat_seq, id=rec.id)
        cat_nter_cc.append(cat_rec)
        already_visited.add(rec.id)
cat_nter_cc_names = set([rec.id for rec in cat_nter_cc])
cat_len = cat_nter_cc.get_alignment_length()
print(len(cat_nter_cc))
# Append sequences which are present in cc_al but have no N-Terminal domain
len_nter = nter_al.get_alignment_length()
len_cat_nter_cc = cat_nter_cc.get_alignment_length()
for rec in tqdm.tqdm(cc_al):
    if not rec.id in cat_nter_cc_names:
        cat_seq = "".join(["-" for k in range(len_nter)]) + rec.seq
        cat_rec = SeqRecord(seq=cat_seq, id=rec.id)
        cat_nter_cc.append(cat_rec)
        cat_nter_cc_names.add(rec.id)

AlignIO.write(cat_nter_cc, "data/cat_n-ter_coiled-coil_with-ruvc-like.afa", "fasta")

# Concatenate [N-Ter, CC] alignment with C-Terminal alignment
cat_nter_cc_cter = MultipleSeqAlignment([])
already_visited = set()
cter_len = cter_al.get_alignment_length()
for rec in tqdm.tqdm(cter_al):
    if not rec.id in already_visited:
        if rec.id in cat_nter_cc_names:
            add_rec = get_record(cat_nter_cc, rec.id)
            cat_seq = add_rec.seq + rec.seq  # append C-Ter to the right
            cat_rec = SeqRecord(seq=cat_seq, id=rec.id)
        else:
            cat_seq = "".join(["-" for i in range(cat_len)]) + rec.seq  # append gaps to the left/C-Ter to the right
            cat_rec = SeqRecord(seq=cat_seq, id=rec.id)
        cat_nter_cc_cter.append(cat_rec)
        already_visited.add(rec.id)
cat_nter_cc_cter_names = set([rec.id for rec in cat_nter_cc_cter])

# Append sequences which are present in nter_cc_al but have no C-Terminal domain
len_nter_cc = cat_nter_cc.get_alignment_length()
len_nter_cc_cter = cat_nter_cc_cter.get_alignment_length()
for rec in tqdm.tqdm(cat_nter_cc):
    if not rec.id in cat_nter_cc_cter_names:
        cat_seq = rec.seq + "".join(["-" for k in range(cter_al.get_alignment_length())])
        cat_rec = SeqRecord(seq=cat_seq, id=rec.id)
        cat_nter_cc_cter.append(cat_rec)
        cat_nter_cc_cter_names.add(rec.id)

AlignIO.write(cat_nter_cc_cter, "data/cat_n-ter_coiled-coil_c-ter_with-ruvc-like.afa", "fasta")

100%|██████████| 3548/3548 [00:01<00:00, 2172.90it/s]


3520


100%|██████████| 3903/3903 [00:04<00:00, 895.50it/s] 


1

In [25]:
nter_cc_names = set()
with open("data/cat_n-ter_coiled-coil.afa", "r") as f:
    nter_cc_al = AlignIO.read(f, "fasta")
    for rec in nter_cc_al:
        id_short = rec.id.split(":")[0]
        rec.id = id_short
        nter_cc_names.add(rec.id)

nter_cc_cter_names = set()
with open("data/cat_n-ter_coiled-coil_c-ter.afa", "r") as f:
    nter_cc_cter_al = AlignIO.read(f, "fasta")
    for rec in nter_cc_cter_al:
        id_short = rec.id.split(":")[0]
        rec.id = id_short
        nter_cc_cter_names.add(rec.id)

print(len(nter_cc_al), len(nter_cc_cter_al))

3227 3274
